1.encoding

In [ ]:
from sklearn.preprocessing import LabelEncoder
import pandas as pd

# To read the dataset from a CSV file
df = pd.read_csv('Original_dataset.csv')

# To create a new DataFrame for encoded values
encoded_df = df.copy()

# Initialize LabelEncoder
label_encoder = LabelEncoder()


# Iterate through each column in the original DataFrame
for column in df.columns:
    # To Check if the column has string values (categorical)
    if df[column].dtype == 'object':
        # LabelEncoder to transform the categorical values to numerical values
        encoded_df[column] = label_encoder.fit_transform(df[column])

# Save the encoded DataFrame to a new CSV file
encoded_df.to_csv('encoded_best_data.csv', index=False)

# Display the original and encoded DataFrames
print("Original DataFrame:")
print(df)
print("\nEncoded DataFrame:")
print(encoded_df)


Original DataFrame:
       duration protocol_type   service  flag  src_bytes  dst_bytes  land  \
0             0           tcp  ftp_data    SF        491          0     0   
1             0           udp     other    SF        146          0     0   
2             0           tcp   private    S0          0          0     0   
3             0           tcp      http    SF        232       8153     0   
4             0           tcp      http    SF        199        420     0   
...         ...           ...       ...   ...        ...        ...   ...   
95946         0           tcp   private   REJ          0          0     0   
95947         0           tcp      http    SF        254       2277     0   
95948         0           tcp   private    S0          0          0     0   
95949         0           tcp   private  RSTR          0          0     0   
95950         0           udp  domain_u    SF         44        129     0   

       wrong_fragment  urgent  hot  ...  dst_host_same_

2.to split the dataset into two

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split

# Load your dataset (replace 'your_dataset.csv' with the actual filename and path)
df = pd.read_csv('encoded_best_data.csv')

# Assuming 'attack_method' is the column containing the target variable
X = df.drop('attack_method', axis=1)  # Features
y = df['attack_method']  # Target variable

# Split the dataset into 85% training and 20% testing
df_train, df_test = train_test_split(df, test_size=0.20, random_state=42)

# Optionally, you can save these datasets to new CSV files
df_train.to_csv('encoded_best_train_data.csv', index=False)
df_test.to_csv('encoded_best_test_data.csv', index=False)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

3. To count labels

In [ ]:

df = pd.read_csv('encoded_best_train_data.csv')

# Count and display the number of labels in attack_method
if 'attack_method' in df:
    print("\nCount of labels in 'attack_method':")
    label_counts = df['attack_method'].value_counts()
    print(label_counts)
    print(f"\nThere are {len(label_counts)} unique labels in the 'attack_method' column.")
else:
    print("\n'attack_method' column was not found in the DataFrame.")


Count of labels in 'attack_method':
1    40318
0    27965
2     7148
3     1197
4       78
5       54
Name: attack_method, dtype: int64

There are 6 unique labels in the 'attack_method' column.


4.gan

final problem

In [ ]:
import pandas as pd
!pip install --upgrade ctgan
from ctgan import CTGAN
from sklearn.utils import shuffle

# Function to apply non-negativity constraint and round decimals
def clean_data(data):
    return data.clip(lower=0).round(2)

# Load the original dataset and ensure no label 5
real_data = pd.read_csv('/content/encoded_best_train_data.csv')
real_data = real_data[real_data['attack_method'] != 5]

# Define the desired final counts for each label, including label 1 as is
desired_counts = {
    0: 9763,
    1: real_data[real_data['attack_method'] == 1].shape[0],  # Keep label 1 as is
    2: 2791 + real_data[real_data['attack_method'] == 2].shape[0],
    3: 7967 + real_data[real_data['attack_method'] == 3].shape[0],
    4: 9715 + real_data[real_data['attack_method'] == 4].shape[0],
}

# Initialize the CTGAN model
ctgan_model = CTGAN(verbose=True)

# Prepare a dataframe to collect all processed data
processed_data = pd.DataFrame()

for label, desired_count in desired_counts.items():
    # Existing count in real_data
    current_count = real_data[real_data['attack_method'] == label].shape[0]

    # Check if the label needs upsampling
    if current_count < desired_count:
        # Calculate the number of synthetic samples needed
        synthetic_needed = desired_count - current_count

        # Filter the training data for the current label
        training_data = real_data[real_data['attack_method'] == label].copy()

        # Fit the CTGAN model on the current label's data
        ctgan_model.fit(training_data)

        # Generate synthetic samples and clean them
        synthetic_data = ctgan_model.sample(synthetic_needed)
        synthetic_data = clean_data(synthetic_data)  # Apply non-negativity and rounding
        synthetic_data['attack_method'] = label  # Ensure correct label assignment

        # Append both original and synthetic data for the current label
        processed_data = pd.concat([processed_data, training_data, synthetic_data])
    else:
        # For labels that do not need synthetic data, or need downsampling
        processed_data = pd.concat([processed_data, real_data[real_data['attack_method'] == label].sample(desired_count, random_state=42)])

# Re-indexing after concatenation
processed_data.reset_index(drop=True, inplace=True)

# Shuffle the dataset to mix original and synthetic samples evenly
processed_data = shuffle(processed_data, random_state=42)

# Ensure the final dataframe adheres to non-negativity and rounding, if not already done
processed_data = clean_data(processed_data)

# Save the combined dataset to a new CSV file
processed_data.to_csv('final_multi_balanced_data.csv', index=False)

print("Augmented balanced dataset with original and synthetic data created and saved.")


Gen. (-1.47) | Discrim. (-0.82): 100%|██████████| 300/300 [26:25<00:00,  5.28s/it]
Gen. (-1.58) | Discrim. (-0.30): 100%|██████████| 300/300 [01:29<00:00,  3.35it/s]
Gen. (-1.24) | Discrim. (0.36): 100%|██████████| 300/300 [00:53<00:00,  5.57it/s]


Augmented balanced dataset with original and synthetic data created and saved.


In [ ]:
import pandas as pd
df = pd.read_csv('augmented_balanced_data_new.csv')

# Count and display the number of labels in attack_method
if 'attack_method' in df:
    print("\nCount of labels in 'attack_method':")
    label_counts = df['attack_method'].value_counts()
    print(label_counts)
    print(f"\nThere are {len(label_counts)} unique labels in the 'attack_method' column.")
else:
    print("\n'attack_method' column was not found in the DataFrame.")


Count of labels in 'attack_method':
1    40318
0    10079
3    10079
2    10079
4    10079
Name: attack_method, dtype: int64

There are 5 unique labels in the 'attack_method' column.
